In [1]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import  models
from visdom import Visdom

In [2]:
# 自定义数据集
class MyDataset(Dataset):
    def __init__(self, dataframe, root_dir, transform=None):
        self.questionnaire_data = dataframe
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.questionnaire_data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,"patient"+str(self.questionnaire_data.iloc[idx, 0]),self.questionnaire_data.iloc[idx, 1])
        image = io.imread(img_name)
        questionnaire = self.questionnaire_data.iloc[idx, 3:].tolist()
        label = self.questionnaire_data.iloc[idx, 2]
        sample = {'image': image, 'questionnaire': questionnaire, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [3]:
# 定义Transforms变换

class ToTensor(object):
    def __call__(self, sample):
        image = sample['image']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1)).copy()
        return {'image': torch.from_numpy(image).float(), 'questionnaire': torch.Tensor(sample['questionnaire']), 'label':sample['label']}

In [4]:
data_df = pd.read_csv("processed_data.csv", index_col=0)
train_df = data_df.sample(frac=0.8,random_state=0,axis=0)
test_df = data_df[~data_df.index.isin(train_df.index)]
train_dataset = MyDataset(train_df,"patient",transform=ToTensor())
test_dataset = MyDataset(test_df,"patient",transform=ToTensor())
train_dataloader =DataLoader(train_dataset, batch_size=1,shuffle=True, num_workers=0)
test_dataloader =DataLoader(test_dataset, batch_size=10,shuffle=True, num_workers=0)


In [5]:
class JointNet(nn.Module):
    def __init__(self,feature_extract=True, num_classes=3, hidden1=2048, hidden2=512, dropout=0.3):
        super(JointNet, self).__init__()
        model = models.vgg16(pretrained=True)
        self.features = model.features
        set_parameter_requires_grad(self.features, feature_extract)#固定特征提取层参数
        self.avgpool=model.avgpool
        self.hidden = nn.Sequential(
            nn.Linear(512*7*7 , hidden1),
            nn.ReLU(),
            nn.Linear(hidden1 , hidden2),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden2+37, hidden2+37),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(hidden2+37, num_classes)
        )
        
    def forward(self, x):
        img, quest = x
        img = self.features(img)
        img = self.avgpool(img)
        img = img.view(img.size(0), 512*7*7)
        img = self.hidden(img)
        joint = torch.cat([img, quest],1)
        out=self.classifier(joint)
        return out
    
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=JointNet(feature_extract=False).to(device)
learning_rate=0.001
num_epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
viz = Visdom()
viz.line([0.], [0], win='train_loss', opts=dict(title='train_loss'))
viz.line([[0.,0.]], [0], win='acc', opts=dict(title='new_acc', legend=['train', 'test']))

Setting up a new session...


'acc'

In [8]:
global_step = 0
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    for sample_batched in train_dataloader:
        data = [sample_batched['image'].to(device),sample_batched['questionnaire'].to(device)]
        labels = sample_batched['label'].to(device)
    

        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        

    model.eval()
    with torch.no_grad():
        for sample_batched in test_dataloader:
            data = [sample_batched['image'].to(device),sample_batched['questionnaire'].to(device)]
            labels = sample_batched['label'].to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            
    print("epoch%d: loss%.5f, train_acc%.5f, test_acc%.5f" % (epoch, loss.item(),train_correct/train_total, test_correct/test_total))
    viz.line([loss.item()], [epoch], win='train_loss', update='append')
    viz.line([[train_correct/train_total,test_correct/test_total]], [epoch], win='acc', update='append')

epoch0: loss0.28765, train_acc0.65611, test_acc0.77273
epoch1: loss0.06962, train_acc0.80090, test_acc0.90909
epoch2: loss0.81688, train_acc0.85294, test_acc0.91818
epoch3: loss0.02896, train_acc0.87783, test_acc0.90000
epoch4: loss0.16453, train_acc0.87783, test_acc0.90000
epoch5: loss0.17567, train_acc0.89593, test_acc0.94545
epoch6: loss0.06124, train_acc0.89819, test_acc0.93636
epoch7: loss0.02106, train_acc0.91176, test_acc0.94545
epoch8: loss0.00261, train_acc0.92760, test_acc0.93636
epoch9: loss1.09894, train_acc0.90950, test_acc0.92727
epoch10: loss0.07256, train_acc0.92534, test_acc0.97273
epoch11: loss0.00171, train_acc0.93665, test_acc0.97273
epoch12: loss0.00010, train_acc0.92081, test_acc0.95455
epoch13: loss0.00152, train_acc0.94118, test_acc0.96364
epoch14: loss0.00384, train_acc0.94796, test_acc0.99091
epoch15: loss0.03360, train_acc0.93439, test_acc0.98182
epoch16: loss0.00002, train_acc0.93891, test_acc0.97273
epoch17: loss0.01006, train_acc0.95475, test_acc1.00000
ep

In [9]:
torch.save(model.state_dict(), "jointmodel.pth")